<a href="https://colab.research.google.com/github/Arka1212/Airline-Passenger-Referral-Prediction/blob/main/Airline_Passenger_Referral_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Data includes airline reviews from 2006 to 2019 for popular airlines around the world with multiple choice and free text questions. Data is scraped in Spring 2019. The main objective is to predict whether passengers will refer the airline to their friends.***

# **Feature descriptions briefly as follows:**

## **airline:** Name of the airline.
## **overall:** Overall point is given to the trip between 1 to 10.
## **author:** Author of the trip
## **reviewdate:** Date of the Review customer review: Review of the customers in free text format
## **aircraft:** Type of the aircraft
## **travellertype:** Type of traveler (e.g. business, leisure)
## **cabin:** Cabin at the flight date flown: Flight date
## **seatcomfort:** Rated between 1-5
## **cabin service:** Rated between 1-5
## **foodbev:** Rated between 1-5 entertainment: Rated between 1-5
## **groundservice:** Rated between 1-5
## **valueformoney:** Rated between 1-5
## **recommended:** Binary, target variable.




In [1]:
# Importing necessary libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
import scipy.stats as stat
from datetime import *
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Mounting drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Reading dataset.
path = '/content/drive/MyDrive/Capstone Projects/Machine Learning (Classification)/Airline Passenger Referral Prediction/data_airline_reviews.xlsx'
df = pd.read_excel(path)